In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
print(os.listdir("../input"))


# Any results you write to the current directory are saved as output.

['mobilefull', 'beauty-nlp-data', 'mobile-nlp-data', 'fashion-nlp-data', 'beauty-nlp-model', 'fashionfull', 'ndsc-beginner', 'fashion-meta-model', 'mobile-meta-model']


In [2]:
def split_groups(df):
    beauty = df[df.image_path.str.contains('beauty')]
    fashion = df[df.image_path.str.contains('fashion')]
    mobile = df[df.image_path.str.contains('mobile')]
    
    return beauty, fashion, mobile

In [3]:
# CNN base model

from keras.models import Model
from keras.applications import Xception
from keras.layers import Dense, Dropout

def get_cnn(num_classes, weights = None):
    if weights:
        base = Xception(include_top = False, pooling = 'avg', classes = num_classes, weights = None)
    else:
        base = Xception(include_top = False, pooling = 'avg', classes = num_classes)

    x = base.output
    x = Dense(1024, activation = 'relu')(x)
    x = Dropout(0.3)(x)
    predictions = Dense(num_classes, activation = 'softmax')(x)
    
    cnn = Model(input = base.input, output = predictions)
    if weights: cnn.load_weights(weights)
        
    cnn.compile(loss = 'categorical_crossentropy',
              optimizer = 'adam',
              metrics = ['accuracy'])
    
    return cnn

Using TensorFlow backend.


In [4]:
# NLP base model

from keras.models import Model
from keras.layers import Input, Dense, Dropout

def get_nlp(num_classes, num_features = 384, weights = None):
    base = Input(shape = (num_features,))
    x = Dense(1024, activation = 'relu')(base)
    x = Dropout(0.3)(x)
    predictions = Dense(num_classes, activation = 'softmax')(x)

    nlp = Model(input = base, output = predictions)
    if weights: nlp.load_weights(weights)

    nlp.compile(loss = 'categorical_crossentropy',
              optimizer = 'adam',
              metrics = ['accuracy'])

    return nlp

In [5]:
# Meta classifier

from keras.models import Model
from keras.layers import concatenate

def get_meta(base_models, num_classes, weights = None):
    x = concatenate([base.output for base in base_models])
    x = Dense(128, activation = 'relu')(x)
    x = Dropout(0.3)(x)
    predictions = Dense(num_classes, activation = 'softmax')(x)

    meta = Model(input = [base.input for base in base_models], output = predictions)
    if weights: meta.load_weights(weights)

    meta.compile(loss = 'categorical_crossentropy',
               optimizer = 'adam',
               metrics = ['accuracy'])

    return meta

In [6]:
# Convenience constructor

def get_models(num_classes, weights = dict()):
    cnn = get_cnn(num_classes = num_classes, weights = weights.get('cnn', None))
    nlp = get_nlp(num_classes = num_classes, weights = weights.get('nlp', None))
    meta = get_meta([cnn, nlp], num_classes = num_classes, weights = weights.get('meta', None))
    
    return cnn, nlp, meta

In [7]:
# Search directory for the best model weights

import os
import re

pattern = r'(?P<name>.*?)_(?P<val_loss>.*?)-(?P<val_acc>.*?)\.hdf5'
def get_best_weights(directory, pattern = pattern):
    best_loss = None
    best_weights = None
    
    for weights in os.listdir(directory):
        match = re.match(pattern, weights)
        if match:
            val_loss = float(match.group('val_loss'))
            if (best_loss == None) or (val_loss <= best_loss):
                best_loss = val_loss
                best_weights = weights
    
    if best_weights:
        return directory + best_weights
    else:
        return None

In [8]:
# Data generator

import spacy
from keras.utils import Sequence
from keras.applications.xception import preprocess_input
from skimage.io import imread
from skimage.transform import resize
from sklearn.preprocessing import label_binarize

class MetaDataGenerator(Sequence):
    def __init__(self, dataframe, directory, docvec, classes, batch_size = 32):
        self.image_dir = directory        
        self.image_name = dataframe.image_name
        self.docvec = docvec
        self.category = classes
        self.batch_size = batch_size
        
    def __len__(self):
        return int(np.ceil(self.image_name.shape[0] / self.batch_size))
    
    def __getitem__(self, index):
        lo = self.batch_size * index
        hi = self.batch_size * index + self.batch_size
        
        cnn_input = self.image_name[lo:hi].apply(lambda f: imread(self.image_dir + f))
        cnn_input = np.vstack([preprocess_input(img[np.newaxis,..., 0:3]) for img in cnn_input.values])
        nlp_input = self.docvec[lo:hi]
        
        if isinstance(self.category, np.ndarray):
            return [cnn_input, nlp_input], self.category[lo:hi]
        else:
            return [cnn_input, nlp_input]

## Load data

In [9]:
meta = pd.read_json('../input/ndsc-beginner/categories.json')
beauty_class = [str(int(k)) for k in meta.Beauty.dropna().unique()]
fashion_class = [str(int(k)) for k in meta.Fashion.dropna().unique()]
mobile_class = [str(int(k)) for k in meta.Mobile.dropna().unique()]

## Load models

In [10]:
beauty_weights = get_best_weights('../input/beauty-nlp-model/')
beauty_cnn, beauty_nlp, beauty_meta = get_models(num_classes = len(beauty_class), weights = {'nlp': beauty_weights})

fashion_weights = get_best_weights('../input/fashion-meta-model/')
fashion_cnn, fashion_nlp, fashion_meta = get_models(num_classes = len(fashion_class), weights = {'meta': fashion_weights})

mobile_weights = get_best_weights('../input/mobile-meta-model/')
mobile_cnn, mobile_nlp, mobile_meta = get_models(num_classes = len(mobile_class), weights = {'meta': mobile_weights})

83689472/83683744 [==============================] - 1s 0us/step


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:18: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:12: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  if sys.path[0] == '':
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:12: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("de...)`
  if sys.path[0] == '':


## Generate predictions for each group

In [11]:
beauty_Xte = np.load('../input/beauty-nlp-data/X_test.npy')
beauty_pred = beauty_nlp.predict(beauty_Xte).argmax(axis=1)

In [12]:
fashion_testgen = MetaDataGenerator(
    dataframe = pd.read_csv('../input/fashionfull/fashion_test.csv'),
    directory = '../input/fashionfull/fashion_image_resized/fashion_image_resized/test/',
    docvec = np.load('../input/fashion-nlp-data/X_test.npy'),
    classes = None,
)

fashion_pred = fashion_meta.predict_generator(fashion_testgen).argmax(axis=1)

In [13]:
# fashion_Xte = np.load('../input/fashion-nlp-data/X_test.npy')
# fashion_pred = fashion_nlp.predict(fashion_Xte).argmax(axis=1)

In [14]:
mobile_testgen = MetaDataGenerator(
    dataframe = pd.read_csv('../input/mobilefull/mobile_test.csv'),
    directory = '../input/mobilefull/mobile_image_resized/mobile_image_resized/test/',
    docvec = np.load('../input/mobile-nlp-data/X_test.npy'),
    classes = None,
)

mobile_pred = mobile_meta.predict_generator(mobile_testgen).argmax(axis=1)

In [15]:
# mobile_Xte = np.load('../input/mobile-nlp-data/X_test.npy')
# mobile_pred = mobile_nlp.predict(mobile_Xte).argmax(axis=1)

## Aggregate group predictions into submission csv

In [16]:
beauty, fashion, mobile = split_groups(pd.read_csv('../input/ndsc-beginner/test.csv'))

predictions = pd.DataFrame({
    'itemid': np.hstack([beauty.itemid.values,
                         fashion.itemid.values, 
                         mobile.itemid.values]),
    'Category': [*[beauty_class[x] for x in beauty_pred],
                 *[fashion_class[x] for x in fashion_pred],
                 *[mobile_class[x] for x in mobile_pred]]
})

predictions = pd.read_csv('../input/ndsc-beginner/test.csv').merge(predictions)
predictions.to_csv('combined-meta-model.csv', columns = ['itemid', 'Category'], index = False)